<a href="https://colab.research.google.com/github/sekihiro/Colabo/blob/master/bert_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorchとtransformersを用いて日本語の文章を分類するclassifierを作成、ファインチューニングして予測する
- https://qiita.com/nekoumei/items/7b911c61324f16c43e7e
- https://github.com/nekoumei/DocumentClassificationUsingBERT-Japanese

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# MeCab
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3

# transformers(旧名：pytorch-pretrained-BERT)
!pip install transformers

# DocumentClassifierUsingBertJapanese用
!pip install mojimoji
!pip install logzero

In [3]:
!nvidia-smi
!nvcc -V
!cat /usr/include/cudnn.h | grep CUDNN_MAJOR -A 2

Sun Feb 16 02:49:49 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.48.02    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [7]:
# importの対象ディレクトリのパス設定
import sys
print(sys.path)

NB_PATH = '/content/drive/My Drive/Colab Notebooks'
sys.path.append(NB_PATH)
print(sys.path)

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Colab Notebooks', '/content/drive/My Drive/Colab Notebooks']
['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython', '/content/drive/My Drive/Colab Notebooks', '/content/drive/My Drive/Colab Notebooks', '/content/drive/My Drive/Colab Notebooks']


In [8]:
import os
import re
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import random
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import time
from glob import glob
from tqdm import tqdm
import pprint
import linecache
import MeCab
import cloudpickle
import shutil
%matplotlib inline
from transformers import BertJapaneseTokenizer, BertForSequenceClassification
import DocumentClassifierUsingBertJapanese as clf

In [9]:
# 再現性確保
def seed_everything(seed=1234):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    return seed

# 再現性確保
ret = seed_everything(1234)
print('seed : {}'.format(ret))

# select device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('device : {}'.format(device))

seed : 1234
device : cuda


### livedoor ニュースコーパス 取得
- https://www.rondhuit.com/download/ldcc-20140209.tar.gz

In [10]:
%cd "/content/drive/My Drive/git/LDCC/data/text/"
%ls -al

/content/drive/My Drive/git/LDCC/data/text
total 40
drwx------ 2 root root 4096 Feb  9  2014 dokujo-tsushin/
drwx------ 2 root root 4096 Feb 15 15:02 .ipynb_checkpoints/
drwx------ 2 root root 4096 Feb  9  2014 it-life-hack/
drwx------ 2 root root 4096 Feb  9  2014 kaden-channel/
drwx------ 2 root root 4096 Feb  9  2014 livedoor-homme/
drwx------ 2 root root 4096 Feb  9  2014 movie-enter/
drwx------ 2 root root 4096 Feb  9  2014 peachy/
drwx------ 2 root root 4096 Feb  9  2014 smax/
drwx------ 2 root root 4096 Feb  9  2014 sports-watch/
drwx------ 2 root root 4096 Feb  9  2014 topic-news/


In [11]:
# テキスト内容を確認
!head "/content/drive/My Drive/git/LDCC/data/text/it-life-hack/it-life-hack-6292880.txt"

http://news.livedoor.com/article/detail/6292880/
2012-02-19T13:00:00+0900
旧式Macで禁断のパワーアップ！最新PCやソフトを一挙にチェック【ITフラッシュバック】
テレビやTwitterと連携できるパソコンや、プロセッサや切り替わるパソコンなど、面白いパソコンが次から次へと登場した。旧式Macの禁断ともいえるパワーアップ方法から、NECの最新PC、話題のThinkPad X1 Hybrid、新セキュリティソフトまで一挙に紹介しよう。

■インテル SSD 520をMacに装着！旧式Macはどれほど高速化するのか (上)
インテルが最新SSD「520シリーズ」を発売した。現行SSDの中でもトップクラスの性能を誇る同製品を、旧型Macの高速化を図るというポイントでレビューしてみた。少し風変わりなレビューとなるが、どの程度の効果があるか、期待大である。


■http://itlifehack.jp/archives/6716997.html


In [0]:
BASE_PATH = "/content/drive/My Drive/git/LDCC/"
DATA_PATH = BASE_PATH + 'data/text/'
PICKLE_PATH = BASE_PATH + 'data/ldcc_idx.pkl' # ラベルカラムは、文字列でなく、インデックスNo

In [13]:
# LDCCテキストファイルからタイトル行をDateFrameに格納し、pickle形式で保存

# カテゴリを配列で取得
categories = []
list_cat = glob(DATA_PATH + '*')
for tmp in list_cat:
    categories.append(tmp[tmp.rindex('/')+1:])
#print(categories)
time.sleep(1)
# ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']
dic = {}
for i, buf in enumerate(categories):
    dic[buf] = i
pprint.pprint(dic)

if os.path.isfile(PICKLE_PATH):
    print(PICKLE_PATH + ' is exists ... ')
else:
    # データフレーム作成
    datasets = pd.DataFrame(columns=['Text', 'Label'])
    for cat in categories:
        path = DATA_PATH + cat + "/*.txt"
        files = glob(path)
        print('\n' + cat + '\n')
        time.sleep(1)
        for text_name in tqdm(files):
            # 各テキストファイルの3行目にタイトル文字列の記載がある
            title = linecache.getline(text_name, 3)
            s = pd.Series([title, dic[cat]], index=datasets.columns)
            datasets = datasets.append(s, ignore_index=True)

    # データフレームシャッフル
    print('\n len : {}'.format(len(datasets)))
    datasets = datasets.sample(frac=1).reset_index(drop=True)
    datasets.head()

    # PICKLE形式で保存
    datasets.to_pickle(PICKLE_PATH)
    print(PICKLE_PATH + ' is saved ...')

{'.ipynb_checkpoints': 9,
 'dokujo-tsushin': 0,
 'it-life-hack': 7,
 'kaden-channel': 2,
 'livedoor-homme': 1,
 'movie-enter': 6,
 'peachy': 5,
 'smax': 3,
 'sports-watch': 8,
 'topic-news': 4}
/content/drive/My Drive/git/LDCC/data/ldcc_idx.pkl is exists ... 


In [0]:
# PICKLE形式のDataFrameを読み込む
df = pd.read_pickle(PICKLE_PATH)
#datasets.head()

In [15]:
#df = pd.read_csv('path/to/data/livedoor-news-data.tar.gz')
#le = LabelEncoder()
#df['label'] = le.fit_transform(df.label.values)
#df = df[['text', 'label']]
#df.columns = ['Text', 'Label']
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df.Label)
print('train_df len : {}'.format(len(train_df)))
print('val_df   len : {}'.format(len(val_df)))
df.head()

train_df len : 3628
val_df   len : 908


,Text,Label
0,欧州選手権開幕、ストイコビッチ監督の優勝国予想は？\n,8
1,ラジオ英会話1年分を手のひらサイズに収録！SII電子辞書G6シリーズ【売れ筋チェック】\n,2
2,充電中の青い光が美しい！ サンワサプライ「光るiPhone・iPad USBケーブル」【売れ...,2
3,いつまで経っても減らない迷惑メールに新法律が施行\n,0
4,何が何でもiPhoneを守りたい人たちへ！ ハンマーでたたいても大丈夫な保護フィルム「ウルト...,2


In [16]:
SRC_PATH = "/content/drive/My Drive/Colab Notebooks/checkpoint.pt"
DST_PATH = "/content/drive/My Drive/git/model/bert-fine_tuning/checkpoint.pt"
MODEL_PATH_LABEL9 = "/content/drive/My Drive/git/model/bert/base-japanese-whole-word-masking_label9/"

# checkpointファイルの保存場所指定
%cd "/content/drive/My Drive/Colab Notebooks"

/content/drive/My Drive/Colab Notebooks


### ファインチューニング
- BertForSequenceClassification : 分類問題のためのクラス
- ベースとするモデルは、bert-base-japanese-whole-word-masking
- DocumentClassifier.__init__ で BertForSequenceClassification.from_pretrained を呼んでいる
- from_pretrained時にnum_labelsを指定してあげることで、任意のクラス数の分類器ができる (デフォルトは2クラス分類器)


In [20]:
if os.path.isfile(DST_PATH):
    print('{} is exists ... '.format(DST_PATH))

else:
    print('fine tuning ...')

    if os.path.isfile(MODEL_PATH_LABEL9 + 'pytorch_model.bin'):
        print('loading model ... ')
        model = clf.DocumentClassifier(num_labels=9, num_epochs=30, net_dir=MODEL_PATH_LABEL9)
    else:
        print('downloading model ... ')
        model = clf.DocumentClassifier(num_labels=9, num_epochs=20)
        print('saving model ... ')
        model.net.save_pretrained(MODEL_PATH_LABEL9)

    model.fit(train_df, val_df, early_stopping_rounds=10)
    shutil.move(SRC_PATH, DST_PATH)

    y_proba = model.predict(val_df)
    pprint.pprint(y_proba)

/content/drive/My Drive/git/model/bert-fine_tuning/checkpoint.pt is exists ... 


### 評価

In [23]:
# checkpointファイルを読み込み、再度評価してみる
print('load check point file ... {}'.format(DST_PATH))
model = clf.DocumentClassifier(num_labels=9, num_epochs=30, net_dir=MODEL_PATH_LABEL9)
model.net.load_state_dict(torch.load(DST_PATH))
y_proba = model.predict(val_df)

load check point file ... /content/drive/My Drive/git/model/bert-fine_tuning/checkpoint.pt


<All keys matched successfully>

In [48]:
print(y_proba.shape)
pprint.pprint(y_proba)

(908, 9)
array([[5.9125122e-02, 9.3192816e-01, 1.2986524e-03, ..., 2.0903891e-03,
        5.1427813e-04, 5.9777155e-04],
       [4.7177394e-04, 3.3920135e-02, 2.4679131e-04, ..., 9.5792055e-01,
        2.8135037e-04, 4.5125815e-03],
       [5.9825554e-04, 5.5243075e-04, 1.9002571e-03, ..., 3.8402868e-05,
        1.4844959e-03, 3.3173020e-04],
       ...,
       [3.3444918e-05, 7.8308956e-05, 9.9885988e-01, ..., 2.0157124e-05,
        6.6519465e-04, 6.4269836e-05],
       [8.9946389e-04, 2.9945711e-03, 1.2151452e-04, ..., 9.9249274e-01,
        2.7939957e-04, 1.2534753e-03],
       [1.5380960e-04, 3.0288668e-03, 2.7486405e-03, ..., 2.4835108e-04,
        9.9328929e-01, 6.3886117e-05]], dtype=float32)


### 確認

In [42]:
# 'dokujo-tsushin': 0,
# 'livedoor-homme': 1,
# 'kaden-channel': 2,
# 'smax': 3,
# 'topic-news': 4
# 'peachy': 5,
# 'movie-enter': 6,
# 'it-life-hack': 7,
# 'sports-watch': 8,
val_df.head()

,Text,Label
2822,ビジネスマン必読の新書「ビジネス服薬術」が出版、第一印象と心に効くファッションとは？\n,1
2454,インタビュー：リン・コリンズ「現代にはヒーローが求められている」\n,6
966,NTTドコモ、MEDIAS PP N-01Dにてマナーモード設定時に音声着信を応答保留にする...,3
2819,夏に多発！下着チラみせ独女はアリかナシか？\n,0
2578,北乃きい、東日本大震災が起きて感じたこと\n,6


In [58]:
print('idx\tLabel\tprobability')
for i, j in enumerate(range(0,5)):
    ret = max(enumerate(y_proba[j]), key = lambda x:x[1])
    print('{}\t{}\t{}'.format(i, ret[0], ret[1]))

idx	Label	probability
0	1	0.9319281578063965
1	6	0.9579205513000488
2	3	0.988389253616333
3	0	0.9574654698371887
4	4	0.384769469499588
